In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import acquire

In [2]:
response = requests.get("https://codeup.com/blog/", headers={"user-agent": "Codeup DS"})
soup = BeautifulSoup(response.text)
links = [link.attrs["href"] for link in soup.select(".more-link")]

In [3]:
def get_first_page_post_links():
    response = requests.get("https://codeup.com/blog/", headers={"user-agent": "Codeup DS"})
    soup = BeautifulSoup(response.text)
    links = []
    for link in soup.select(".more-link"):
        if link.get("href") and link.get("href")[0:4] == "http":
            links.append(link.get("href"))
    return links

def parse_post_links(links):
    posts = []
    for link in links:
        response = requests.get(link, headers={"user-agent": "Codeup DS"})
        soup = BeautifulSoup(response.text)
        title = soup.select_one(".entry-title").text
        date = soup.select_one(".published").text
        body = soup.select_one(".entry-content").text.strip()
        posts.append({"title": title, "date": date, "body": body})
    return pd.DataFrame(posts)

def get_all_post_links():
    url = "https://codeup.com/blog/page/{}/?et_blog"
    response = requests.get(url, headers={"user-agent": "Codeup DS"})
    soup = BeautifulSoup(response.text)
    links = []
    for link in soup.select(".more-link"):
        if link.get("href") and link.get("href")[0:4] == "http":
            links.append(link.get("href"))
    return links
    


In [4]:
parse_post_links(get_first_page_post_links())

,title,date,body
0,Learn to Code: Python Workshop on 4/23,"Mar 31, 2022","According to LinkedIn, the “#1 Most Promising ..."
1,Coming Soon: Cloud Administration,"Mar 17, 2022",We’re launching a new program out of San Anton...
2,5 Books Every Woman In Tech Should Read,"Mar 8, 2022",On this International Women’s Day 2022 we want...
3,Codeup Start Dates for March 2022,"Jan 26, 2022",As we approach the end of January we wanted to...
4,VET TEC Funding Now Available For Dallas Veterans,"Jan 7, 2022",We are so happy to announce that VET TEC benef...
5,Dallas Campus Re-opens With New Grant Partner,"Dec 30, 2021",We are happy to announce that our Dallas campu...
6,Codeup’s Placement Team Continues Setting Records,"Nov 19, 2021",Our Placement Team is simply defined as a grou...
7,"IT Certifications 101: Why They Matter, and Wh...","Nov 18, 2021","AWS, Google, Azure, Red Hat, CompTIA…these are..."
8,A rise in cyber attacks means opportunities fo...,"Nov 17, 2021","In the last few months, the US has experienced..."
9,Use your GI Bill® benefits to Land a Job in Tech,"Nov 4, 2021","As the end of military service gets closer, ma..."


In [5]:
parse_post_links(get_all_post_links())

""


In [6]:
def get_all_posts(use_cache=True):
    """ pull all posts from codeup.com  or use the cached version """
    filename = "posts.csv"
    if use_cache and os.path.exists(filename):
        print("Using cached version of posts")
        return pd.read_csv(filename)
    print("Fetching posts from codeup.com")
    page =1
    df = pd.DataFrame()
    url = f"https://codeup.com/blog/page/{page}/?et_blog"
    response = requests.get(url, headers={"user-agent": "Codeup DS"})
    print ("woring on page 1...", end ="")
    while True: # loop until no new links are found
        print (f"\rworking on page {page}...", end ="", flush=True)
        url = f"https://codeup.com/blog/page/{page}/?et_blog"
        response = requests.get(url, headers={"user-agent": "Codeup DS"})
        soup = BeautifulSoup(response.text)
        links = []
        for link in soup.select(".more-link"):
            if link.attrs["href"] and link.get("href")[0:4] == "http":
                links.append(link.get("href"))
        #print(f'links length: {len(links)}')
        if len(links) == 0: #break when no new links are found
            break
        posts = parse_post_links(links)
        df = df.append(posts)
        page += 1
    df.reset_index(inplace=True, drop=True)
    df.to_csv(filename, index=False)
    return df

    

In [11]:
df = acquire.get_all_posts()
df

Using cached version of posts


,title,date,body
0,Learn to Code: Python Workshop on 4/23,"Mar 31, 2022","According to LinkedIn, the “#1 Most Promising ..."
1,Coming Soon: Cloud Administration,"Mar 17, 2022",We’re launching a new program out of San Anton...
2,5 Books Every Woman In Tech Should Read,"Mar 8, 2022",On this International Women’s Day 2022 we want...
3,Codeup Start Dates for March 2022,"Jan 26, 2022",As we approach the end of January we wanted to...
4,VET TEC Funding Now Available For Dallas Veterans,"Jan 7, 2022",We are so happy to announce that VET TEC benef...
...,...,...,...
216,Why Isn’t the San Antonio Tech Scene Growing F...,"Aug 14, 2018",The simple answer is computer programming tale...
217,Why People Can’t Learn Programming on Their Own,"Aug 14, 2018","While developing Codeup, we interviewed dozens..."
218,What is Our Noble Cause?,"Aug 14, 2018",In his TEDx San Antonio presentation this fall...
219,Scholarships for Women: Why We’re Doing It,"Aug 14, 2018",A hot topic that is trending is the special tr...


In [8]:
url = 'https://www.inshorts.com/en/read/business'
response = requests.get(url, headers={'user-agent': 'Codeup DS'})
soup = BeautifulSoup(response.text)
cards = soup.select('.news-card')
len(cards)
card = cards[0]
card.find('span', itemprop = 'headline').text


'Rupee hits all-time low of 77.42 against US dollar'

In [9]:
def parse_news_card(card):
    'Given a news card object, returns a dictionary of the relevant information.'
    card_title = card.select_one('.news-card-title')
    output = {}
    output['title'] = card.find('span', itemprop = 'headline').text
    output['author'] = card.find('span', class_ = 'author').text
    output['content'] = card.find('div', itemprop = 'articleBody').text
    output['date'] = card.find('span', clas ='date').text
    return output


def parse_inshorts_page(url):
    '''Given a url, returns a dataframe where each row is a news article from the url.
    Infers the category from the last section of the url.'''
    category = url.split('/')[-1]
    response = requests.get(url, headers={'user-agent': 'Codeup DS'})
    soup = BeautifulSoup(response.text)
    cards = soup.select('.news-card')
    df = pd.DataFrame([parse_news_card(card) for card in cards])
    df['category'] = category
    return df

def get_inshorts_articles(use_cache=True):
    '''
    Returns a dataframe of news articles from the business, sports, technology, and entertainment sections of
    inshorts.
    '''
    filename = 'inshorts_articles.csv'
    if use_cache and os.path.exists(filename):
        print('Using cached version of inshorts articles')
        return pd.read_csv(filename)
    print('Fetching inshorts articles')
    url = 'https://inshorts.com/en/read/'
    categories = ['business', 'sports', 'technology', 'entertainment']
    df = pd.DataFrame()
    for cat in categories:
        df = pd.concat([df, pd.DataFrame(parse_inshorts_page(url + cat))])
    df = df.reset_index(drop=True)
    df.to_csv(filename, index=False)
    return df

In [10]:
acquire.get_inshorts_articles()


Using cached version of inshorts articles


,title,author,content,date,category
0,Rupee hits all-time low of 77.42 against US do...,Apaar Sharma,The Indian rupee fell to an all-time low of 77...,"09 May 2022,Monday",business
1,Bitcoin falls to the lowest level since Januar...,Pragya Swastik,"Bitcoin fell on Monday to as low as $33,266 in...","09 May 2022,Monday",business
2,Rupee closes at all-time low of 77.50 against ...,Pragya Swastik,The Indian rupee weakened further on Monday to...,"09 May 2022,Monday",business
3,Made best possible decision: IndiGo on barring...,Pragya Swastik,IndiGo's CEO Ronojoy Dutta said the airline ma...,"09 May 2022,Monday",business
4,India's biggest IPO of LIC subscribed nearly 3...,Pragya Swastik,"LIC's IPO, India's biggest IPO which opened on...","09 May 2022,Monday",business
...,...,...,...,...,...
94,"'Prithviraj' educational film, should be shown...",Kriti Kambiri,"Akshay Kumar, speaking at the trailer launch o...","09 May 2022,Monday",entertainment
95,Dileep's wife questioned by Kerala Police in a...,Kriti Kambiri,Malayalam actor Dileep's wife Kavya Madhavan w...,"09 May 2022,Monday",entertainment
96,"Nimrat Kaur shares pic of tattoo, says got it ...",Kriti Kambiri,Actress Nimrat Kaur shared a series of picture...,"09 May 2022,Monday",entertainment
97,I finished my work on 'Prithviraj' in just 42 ...,Udit Gupta,Akshay Kumar has revealed that he completed th...,"09 May 2022,Monday",entertainment
